# E10 — Exercises on Calcium Handling

In this notebook we look at some exercises to explore intracellular calcium handling a bit more hands on. Compared to previous sessions, there is a stronger focus on discussion and work with pen and paper, and a bit less on actual modeling.

This notebook contains the following exercises:

* [Exercise 1: A Simple Calcium Cycling Model](#1)
* [Exercise 2: Thermodynamic Limits of SERCA](#2)
* [Exercise 3: Thermodynamics of the NCX](#3)

<a id="1"></a>
## Exercise 1: A Simple Calcium Cycling Model

We start by implementing a simple calcium cycling model to explore intracellular calcium handling a bit more hands on. The model you implement is based on a work by Friel et al. (1995). The original model was made to explain experimental findings of caffine-induced calcium oscillations in sympatheic neurons. We use the model as propsed by Friel, but change a few of the variable names to be more consistent with how we present the material in L10/E10.

### Introducing the Model

The model proposed by Friel consists of three domains:
* The cytosol, inside the cell
* The sarcoplasmic reticulum (SR), a subdomain inside the cell
* The extracellular space, outside the cell.

We denote the concentration of calcium in each of these domains as $\rm [Ca^{2+}]_{\rm i}$, $\rm [Ca^{2+}]_{\rm SR}$ and $\rm [Ca^{2+}]_{\rm o}$ respectively. The first two of these are model states which change over time, while we assume the extracellular concentration to be constant.

There are four fluxes between these domains:
* There is an external calcium entry through the sarcolemma: $J_{\rm entry}$
* There is an extrusion of calcium through the sarcolemma: $J_{\rm extrusion}$
* There is a release of calcium from the SR: $J_{\rm rel}$
* There is an uptake of calcium back into the SR: $J_{\rm uptake}$.

We can summarize the model with the following figure.

<img src="fig/friel.svg" width=500>

**Exercise 1a) Linear currents**

In the very simplest case, we can assume the four currents to be completely linear, and only dependent on the concentration gradient across a given membrane. For instance, we can then model the influx current as

$$J_{\rm entry} = k_{\rm entry} \left( \rm [Ca^{2+}]_{\rm o} - \rm [Ca^{2+}]_{\rm i} \right).$$

Where $k_{\rm entry}$ would be the *flux rate*, a model parameter of units 1/s. 

* If we assume $\rm [Ca^{2+}]_{\rm o} > \rm [Ca^{2+}]_{\rm i}$, does $J_{\rm entry}$ become positive or negative? 

We also assume the release current to be proportional to the calcium difference between the SR and the cytosol. 

* Write out this current so that it is also positive when entering the cytosol, $J_{\rm rel} = ...$.

Because the uptake and extrusion currents are *active* transport and not passive, let us assume these are proportional to the cytosolic concentration alone, i.e.

$$J_{\rm extrusion} = k_{\rm extrusion} \times [{\rm Ca}^{2+}]_{\rm i}.$$

* Write out the same current for the uptake term, $J_{\rm uptake} = ...$.

**Exercise 1b) Concentration changes**

With the fluxes defined, write out the derivatives of the cytosolic and SR calcium in terms of the four fluxes. Be especially careful that you are consistent in terms of the sign of concentration differences, fluxes and concentration changes.

\begin{equation}
\frac{\rm d [Ca^{2+}]_i}{{\rm d}t} = \ldots,
\end{equation}

\begin{equation}
\frac{\rm d [Ca^{2+}]_{SR}}{{\rm d}t} = \frac{\ldots}{\gamma}.
\end{equation}

Here $\gamma$ is a unitless model parameter needed to account for the fact that the SR has a different volume to the cytosolic space. It has a value of $\gamma = 0.24$, meaning a movement of calcium from the SR to the cytosol will impact the SR concentration roughly four times more than the cytosolic calcium.

**Exercise 1c) Variable release rate**

If you read the original publication, it is not possible to find spontaneous calcium oscillations with purely linear fluxes, as one needs *calcium-induced calcium release* (CICR) to produce proper oscillations. Friel et al., therefore proposes to let the rate constant of the release current $k_{\rm rel}$ be a function of cytosolic calcium, so that it is low when cytosolic calcium is low, but it increases fast if cytosolic calcium grows. 

More specifically, they let

\begin{equation}
k_{\rm rel} = \kappa_0 + \kappa_1 \times \frac{\rm [Ca^{2+}]^n_{\rm i}}{K_{\rm d}^n + [{\rm Ca}^{2+}]_{\rm i}^n}
\end{equation}

What kind of equation is this?

**Exercise 1d) Implementing the RHS**

Fill in the code below to define the RHS of the system

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

def rhs(t, y, Cao, k_entry, k_extrusion, k_uptake, kappa0, kappa1, Kd, n, gamma):
    # Split up the state vector
    Cai, CaSR = y
    
    # Define the linear fluxes
    J_entry = ...
    J_extrusion = ...
    J_uptake = ...
    
    # Compute the release rate (CICR)
    k_rel = ...
    J_rel = ...
    
    # Define the derivatives
    dCai_dt = ...
    dCaSR_dt = (...)/gamma
    
    # Return the derivatives
    return dCai_dt, dCaSR_dt

**Exercise 1e) Solving the model and plotting the solution**

Below we have defined all the parameters of the model. Use these parameters and solve the model for $t\in[0, 1000]$ seconds. As initial conditions use $\rm [Ca]_{\rm i} = 80$ nM and $\rm [Ca]_{\rm SR} = 4.0$ µM. Plot the solutions, $\rm [Ca]_{\rm i}$ and $\rm [Ca]_{\rm SR}$ in two different plots.

In [ ]:
# Parameters
Cao = 1000 # µM

k_entry = 2e-5 # 1/s
k_extrusion = 0.132 # 1/s
k_uptake = 0.9 # 1/s
kappa0 = 0.013 # 1/s
kappa1 = 0.58 # 1/s
Kd = 0.5 # µM
n = 3.
gamma = 0.24

params = (Cao, k_entry, k_extrusion, k_uptake, kappa0, kappa1, Kd, n, gamma)

In [ ]:
# Solve model with solve_ivp
T = ...
y0 = ...

solution = solve_ivp(...)

In [ ]:
# Plot solutions
plt.plot(...)
plt.show(...)

plt.plot(...)
plt.show(...)

**Exercise 1f) Describing the solution**

Explain how the solutions look. You don't have to describe the exact mechanisms of what is going on, but you should explain in broad strokes where calcium is moving in the model. Does it seem reasonable that this is a model to explain a phenomenon called *calcium oscillations*?

**Exercise 1g) Changes in initial conditions (Widget)**

Use the widget below and play around with the initial conditions. Does the initial conditions matter? Why/Why not?

In [ ]:
from L10_widget import CICRWidget

CICRWidget().initial_conditions_widget()

**Exercise 1h) Plotting the CICR rate**

The paper this model is based on is looking at *caffine induced* calcium oscillations. What caffine does is upregulate the release flux's dependency on cytosolic calcium, meaning CICR is stronger. Let us analyze this a bit closer. The release rate was modelled as

\begin{equation}
k_{\rm rel} = \kappa_0 + \kappa_1 \times \frac{\rm [Ca^{2+}]^n_{\rm i}}{K_{\rm d}^n + [{\rm Ca}^{2+}]_{\rm i}^n}
\end{equation}

Plot this curve for $\rm [Ca^{2+}]_{\rm i} \in [0, 2]$ µM.

If caffine makes this function *steeper*, what model parameters could reflect this?





**Exercise 1i) Exploring the CICR model (Widget)**

Below we have made a widget that plots the release rate with adjustable parameters: $\kappa_0$, $\kappa_1$, $K_{\rm d}$, $n$. Play around with this widget and get a feel for what all four parameters do to the rate function. 

If we want a low release current at low calcium, and then a high release rate at high calcium, with a steep and swift shift. What parameters should be low, and which should be high?

In [ ]:
from L10_widget import CICRWidget

CICRWidget().krel_widget()

**Exercise 1j) CICR sweet-spot (Widget)**

In the paper, they state that calcium oscillations are only found at certain levels of caffine. If there is no caffine, then the steepness of the CICR-function ($k_{\rm rel}$ in the model) is too low, and we get no oscillations. If there is too much caffine, then the system releases calcium too readily, and we also get no oscillations.

Attempt to shift either the $K_{\rm d}$ (point of half-saturation) or $n$ (the Hill coefficient) to be very high or low in the widget below. Do the oscillations disappear at either extreme? Hint: The model is a bit heavy, so I suggest you pull the sliders *slowly* to give the plot a chance to update.

In [ ]:
from L10_widget import CICRWidget

CICRWidget().cicr_widget()

<a id="2"></a>
## Exercise 2: Thermodynamic Limits of SERCA

The sarco/endoplasmic reticulum calcium ATPase (SERCA) pump is responsible for pumping calcium into the SR from the cytosol in cardiomyocytes. This has to majorly important functions:
* Maintain a high SR load, which can give a rapid and strong calcium transient through the CICR mechanism
* Maintain very low levels of resting cytosolic calcium, important for allowing the myocyte to relax in between contractions

Ideally the SERCA pump would pump all available calcium from the cytosol into the SR. However, due to the diffusive pressure of calcium, there has to be some theoretical maximum for how large concentration gradients the SERCA pump can achieve. In this exercise you will analyze the thermodynamics of the problem to find this theoretical limit.

**Exercise 2a) Change in Gibbs Energy**

In the lecture on physical chemistry, L2, we introduced a quantity dubbed *Gibbs energy*. Recall now that for any spontaneous process, we must have that $\Delta G < 0$. Explain briefly why this is the case.

For every cycle of the SERCA pump, it pumps two calcium ions from the cytosol into the SR. This will incur a change in Gibbs energy of

$$\Delta G_{\rm pump} = 2RT \ln\left(\frac{\rm [Ca^{2+}]_{SR}}{\rm [Ca^{2+}]_i}\right),$$

It is generally the case that $ {\rm [Ca^{2+}]_{SR}} > {\rm [Ca^{2+}]_i}$. What does this mean for the value of $\Delta G_{\rm pump}$. Explain why this tells us that SERCA must expend energy to move calcium ions into the SR.

**Exercise 2b) Coupling processes**

SERCA is an ATPase, meaning it is an enzyme that can break down ATP molecules into ADP, a process that releases a bit of energy which SERCA can use. Explain why it must be the case that

$$\Delta G_{\rm pump} + \Delta G_{\rm ATP} < 0.$$

Where $G_{\rm ATP}$ is the change in Gibbs Energy we get from "spending" a single ATP molecule.


**Exercise 2c) Maximum concentration ratio**

Insert the expression from 2a into 2b and solve for ${\rm [Ca^{2+}]_{SR}}$. Show that this yields

$${\rm [Ca^{2+}]_{SR}} < {\rm [Ca^{2+}]_{i}} \times {\rm exp}\ \bigg[\frac{\Delta G_{\rm ATP}}{2RT}\bigg].$$

To find a specific value we will need to know the value of $\Delta G_{\rm ATP}$. Looking up the value in the literature reveals that this value changes with several factors. For the conditions inside a mammalian myocyte it will typically be in the range 50–70 kJ/mol. As no molecular machine, like SERCA, will ever be perfectly efficient, it makes sense to pick a number on the lower end of the scale or slightly below this.

Insert $\Delta G_{\rm ATP}$ to find a final estimate. It can be useful to know that for body temperature, $2RT \approx 5.2$ kJ/mol. What is the maximal concentration gradient SERCA can theoretically maintain across the SR membrane?

**Exercise 2d) The difference between theory and practice**

A commonly cited statement is that SERCA can maintain a calcium gradient of about 1:7000, i.e., the SR load can be at most 7000-fold higher than the cytosolic calcium. Compare this statement with the theoretical limit you found. If there are differences, discuss why this might be. Does it seem like SERCA is operating at its thermodynamic limit?

<a id="3"></a>
## Exercise 3: Thermodynamics of the NCX

The sodium-calcium exchanger (NCX) works by transporting 3 sodium ions (Na$^+$) in one direction, and simultaneously transporting 1 calcium ion in the opposite direction (Ca$^{2+}$). Crucially, NCX does not use any other source of energy, like ATP. Let us analyze the thermodynamics of this situation.


**Exercise 3a) Passive or active transport? (Discussion)**

Previously we have introduced the terms *passive* and *active* ion transport. From what you recall of these terms, would you classify the NCX activity as passive or active?

**Exercise 3b) Gibbs energy**

Explain why the change in Gibbs energy of one cycle of the NCX can be described as

$$\Delta G_{\rm NCX} = \Delta G_{\rm Ca} - 3\Delta G_{\rm Na}.$$

**Exercise 3c) Enter the Nernst potentials**

The change in Gibbs energy from moving a particle from inside to outside the cell depends both on the concentration gradient of the ion, but also the membrane potential. We have actually analyzed this exact scenario when we derived the Nernst potential. From that derivation we found that (We did not you expect to remember this expression, which is why we list it here):

$$\Delta G_{\rm X} = z_X F \big( E_{\rm X} - V \big).$$

Here $z_{\rm X}$ is the valency of ionic species X, $E_{\rm X}$ is the Nernst potential of species X, $F$ the Faraday constant and $V$ the membrane potential.

Insert $\Delta G_{\rm Na}$ and $\Delta G_{\rm Ca}$ to find a clean expression for $\Delta G_{\rm NCX}$.

**Exercise 3d) Reversial potential**

We know the system is at thermodynamic equilibrium when $\Delta G = 0$. This in turn means you can find the *reversial* potential of the NCX by solving the equation

$$\Delta G_{\rm NCX} = 0,$$

for the membrane potential $V$. Show that this results in

$$E_{\rm NCX} = 3E_{\rm Na} - 2E_{\rm Ca}.$$

Explain how this potential relates to the forward and reverse modes of the exchanger.

**Exercise 3e) Reversial potential in mV**

In L3 you computed the Nernst potentials under normal conditions and found that $E_{\rm Na} \approx 80$  mV and $E_{\rm Ca} \approx 120$ mV. (Though we expect the calcium Nernst potential to shift considerably during an AP). Insert these values to find $E_{\rm NCX}$ in terms of mV.

**Exercise 3f) The NCX during the action potential**

From your answer. Do we expect to see any reverse mode of NCX during a normal action potential? If so, where in the action potential would we expect to see calcium entering the cell through the NCX? When would we expect calcium to leave the cell?